# F5762 重新架構的 Chat 類別

原本的架構要加入新的工具都要修改類別內的程式，目前的架構只要定義新的工具指令類別，就可以加入新的工具，更具彈性，也更容易講解。

## 匯入通用的模組與建立 OpenAI API 用戶端

In [ ]:
from IPython.display import Markdown, display
from google.colab import userdata
from rich.pretty import pprint
import openai
import sys
import os

In [ ]:
client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

## 第 4 章上傳檔案的輔助函式

In [ ]:
import requests

def upload_file(file_path):
    try:
        if (file_path.startswith('http://')
            or file_path.startswith('https://')):
            response = requests.get(file_path)
            filename = response.headers.get(
                'content-disposition',
                None
            )
            if filename:
                filename = filename.split('filename=')[-1]
                filename = filename.strip('\'"')
            else:
                filename = file_path.split('/')[-1]
            response = client.files.create(
                file=(filename, response.content),
                purpose='user_data'
            )
        else:
            with open(file_path, 'rb') as file:
                response = client.files.create(
                    file=file,
                    purpose='user_data'
                )
    except:
        return None
    return response.id

## 使用者指令處理器的類別

這是基礎類別，訂定了工具指令類別的基本架構：

- handle_command：處理以 '/' 開頭的指令，並依據需要更新 Chat 類別的 tools 工具串列。

    如果傳入的指令是空字串，表示是建立 Chat 類別的物件時設定初始值，若需要在物件一建立就更新它的 tools 串列，就要處理空字串指令。
- handle_event：處理串列輸出時的個別事件：

    - 若還要交給下一個指令處理器，傳回 None
    - 傳回訊息串列表示要送回給模型參考再重新生成
    - 傳回其他內容都表示要交給 Chat 顯示。

In [ ]:
class BaseComand:
    def __init__(self, command, tool_name, icon, verbose=False):
        self.command = command      # 指令
        self.tool_name = tool_name  # 工具名稱
        self.icon = icon            # 工具圖示字元
        self.verbose = verbose      # 啟用詳細輸出
        self.extra_args = {}        # 要額外送給模型的參數

    def handle_command(self, chat, cmd):
        # 不是正確的指令開頭（空字串會是 True）
        if not cmd.startswith(self.command):
            return False
        return True

    def handle_event(self, chat, stream, event):
        return None # 預設不處理交給下一個指令處理器

### 內建搜尋工具的指令處理器

In [ ]:
from urllib.parse import unquote

class WebSearchCommand(BaseComand):
    def __init__(self, verbose=False):
        super().__init__(
            '/w',
            'web_search_preview',
            '🌐',
            verbose
        )

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        idx = chat.find_tool_index(self.tool_name)
        if idx == -1:
            chat.tools.append({
                'type': self.tool_name
            })
        else:
            chat.tools.pop(idx)
        return True

    def show_search_results(self, response):
        if response.output[0].type != "web_search_call":
            return
        content = response.output[1].content[0]
        for i, annotaion in enumerate(
            content.annotations, start=1
        ):
            print(f'{i}. {annotaion.title}')
            print(f'   {unquote(annotaion.url)}')

    def handle_event(self, chat, stream, event):
        if not self.verbose: return None
        if event.type == 'response.completed':
            self.show_search_results(event.response)
        return None

### 內建檔案檢索工具的指令處理器

In [ ]:
class FileSearchCommand(BaseComand):
    def __init__(self, vector_store_id=None, verbose=False):
        super().__init__(
            '/f',
            'file_search',
            '🔍',
            verbose
        )
        self.vector_store_id = vector_store_id

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        idx = chat.find_tool_index(self.tool_name)
        if len(cmd) < 4: # /f[.]，不是冒號加檔名/網址
            if self.vector_store_id == None:
                print('請先使用 /f:[路徑|網址]上傳檔案')
                return True # 沒有向量資料庫無法切換
            turn_on = (idx == -1) # 切換開/關檔案檢索
        else: # /f:檔名|網址，上傳檔案並開啟檢索功能
            turn_on = True
            file_path = cmd[3:]
            file_id= upload_file(file_path)
            if not file_id:
                print(f'無法上傳檔案：{file_path}')
                return True
            if self.vector_store_id == None:
                vector_store = client.vector_stores.create(
                    name="temp",
                    file_ids=[file_id],
                )
                self.vector_store_id = vector_store.id
            else:
                client.vector_stores.files.create(
                    self.vector_store_id,
                    file_id=file_id
                )
        if turn_on:
            chat.tools.append({
                'type': self.tool_name,
                'vector_store_ids': [self.vector_store_id]
            })
            self.extra_args = {
                'include': ['file_search_call.results']
            }
        else:
            chat.tools.pop(idx)
            self.extra_args = {}
        return True

    def remove_vector_store(self, chat):
        if self.vector_store_id == None: return
        idx = chat.find_tool_index('file_search')
        if idx: chat.tools.pop(idx)
        response = client.vector_stores.files.list(
            self.vector_store_id
        )
        for vector_file in response.data:
            client.files.delete(vector_file.id)
        client.vector_stores.delete(self.vector_store_id)

    def show_file_search_results(self, response):
        if response.output[0].type != 'file_search_call':
            return
        results = response.output[0].results
        if not results: return
        for i, result in enumerate(results, start=1):
            display(Markdown('---'))
            print(f'{i}. {result.filename}({result.score})')
            display(Markdown('---'))
            display(Markdown(result.text))

    def handle_event(self, chat, stream, event):
        if not self.verbose: return None
        if event.type == 'response.completed':
            self.show_file_search_results(event.response)
        return None

### 內建程式碼直譯器工具的指令處理器

In [ ]:
class CodeInterpreterCommand(BaseComand):
    def __init__(self, verbose=False):
        super().__init__(
            '/c',
            'code_interpreter',
            '🐍',
            verbose
        )

    def get_file_content_from_container(
            self, container_id, file_id
    ):
        url = (
            'https://api.openai.com/v1/containers/'
            f'{container_id}'
            '/files/'
            f'{file_id}'
            '/content'
        )
        response = requests.get(
            url,
            headers={
                'Authorization': f'Bearer {client.api_key}'
            }
        )
        if response.status_code != 200:
            return None
        return response.content

    def get_code_interpreter_files(self, response):
        files = []
        for output in response.output:
            if output.type != 'message':
                continue
            for content in output.content:
                for annotation in content.annotations:
                    if annotation.type != (
                        'container_file_citation'
                    ):
                        continue
                    files.append((
                        annotation.container_id,
                        annotation.file_id,
                        annotation.filename
                    ))
                    content = (
                        self.get_file_content_from_container(
                        annotation.container_id,
                        annotation.file_id
                    ))
                    if not content:
                        print(f'無法下載 {annotation.filename}')
                        continue
                    with open(annotation.filename, 'wb') as f:
                        f.write(content)
        return files

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        file_ids = []
        if len(cmd) > 3:
            file_id= upload_file(cmd[3:])
            if not file_id:
                print(f'無法上傳檔案：{file_path}')
                return True
            file_ids.append(file_id)
        idx = chat.find_tool_index(self.tool_name)
        if idx == -1:
            chat.tools.append({
                'type': self.tool_name,
                'container': {
                    'type': 'auto',
                    'file_ids': file_ids
                }
            })
            self.extra_args = {
                'include': ['code_interpreter_call.outputs'],
            }
        else:
            chat.tools.pop(idx)
            self.extra_args = {}
        return True

    def handle_event(self, chat, stream, event):
        if event.type == 'response.completed':
            files = self.get_code_interpreter_files(
                event.response
            )
            for f in files:
                print(f'已下載：{f[2]}')
            return None
        if not self.verbose: return None
        if event.type == (
            'response.code_interpreter_call.'
            'in_progress'
        ):
            if stream: return '\n```python\n'
        elif event.type == (
            'response.'
            'code_interpreter_call_code.delta'
        ):
            if stream: return event.delta
        elif event.type == (
            'response.'
            'code_interpreter_call_code.done'
        ):
            if stream: return '\n```\n\n'
            else: return f'\n\n```python\n{event.code}\n```\n\n'
        elif event.type == 'response.output_item.done':
            if event.item.type == 'code_interpreter_call':
                results = (
                    f'\n結果是：\n\n```\n'
                    f'{event.item.outputs[0]["logs"]}'
                    f'\n```\n\n'
                )
                return results
        return None

### 內建影像生成工具的指令處理器

In [ ]:
from IPython.display import Image
import base64

class ImageGenerationCommand(BaseComand):
    def __init__(self, verbose=False):
        super().__init__(
            '/i',
            'image_generation',
            '🎨',
            verbose=verbose,
        )

    def get_img_b64(self, outputs):
        img_data = [
            output.result for output in outputs
            if output.type == 'image_generation_call'
        ]
        return img_data[0] if img_data else None

    def get_img_obj(self, img_b64, width=300):
        return Image(base64.b64decode(img_b64), width=width)

    def save_img(self, img_b64, filename='gen.png'):
        with open(filename, 'wb') as f:
            f.write(base64.b64decode(img_b64))

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        idx = chat.find_tool_index(self.tool_name)
        if idx == -1:
            chat.tools.append({
                'type': self.tool_name,
                'partial_images': 3 if self.verbose else 1
            })
        else:
            chat.tools.pop(idx)
        return True

    def handle_event(self, chat, stream, event):
        if event.type == 'response.completed':
            b64_data = self.get_img_b64(event.response.output)
            if not b64_data: return None
        elif event.type == (
            'response.image_generation_call.partial_image'
        ):
            b64_data = event.partial_image_b64
        else: return None
        return self.get_img_obj(b64_data)

### functional calling 的指令處理器

In [ ]:
from pydantic import BaseModel, Field, ConfigDict
import subprocess

def shell_helper(dangerous, shell_command):

    if dangerous:
        return '指令不安全，無法執行'

    # 啟動子行程
    process = subprocess.Popen(
        shell_command,
        shell=True,             # 在 shell 中執行
        stdout=subprocess.PIPE, # 擷取標準輸出
        stderr=subprocess.PIPE, # 擷取錯誤輸出
        text=True               # 以文字形式返回
    )

    result = '執行結果：\n\n```\n'

    # 即時讀取輸出
    while True:
        output = process.stdout.readline()
        # 如果沒有輸出且行程結束
        if output == '' and process.poll() is not None:
            break
        if output:
            result += output

    result += "```"

    # 檢查錯誤輸出
    error = process.stderr.read()
    if error:
        result += f"\n\n錯誤: {error}"

    # 等待行程結束並取得返回碼
    return_code = process.wait()
    result += f"\n\n命令執行完成，返回碼: {return_code}\n\n"
    return result

class ShellHelper(BaseModel):
    dangerous: bool = Field(
        description='要執行的 shell 指令是否具有危險性？'
    )
    shell_command: str = Field(
        description='要執行的 shell 指令'
    )

shell_helper_tool = {
    'type': 'function',
    "name": "shell_helper",
    "description": "我可以執行 shell 指令操控電腦",
    "parameters": ShellHelper.model_json_schema(),
}

In [ ]:
import json

class FunctionCallingCommand(BaseComand):
    def __init__(self, tools=None, verbose=False):
        super().__init__(
            '/t',
            'function',
            '',
            verbose
        )
        self.tools = tools or []
        self.enabled = False

    def handle_command(self, chat, cmd):
        if not super().handle_command(chat, cmd):
            return False
        if not self.enabled: # 加入自訂函式工具
            chat.tools.extend(self.tools)
        else: # 移除自訂工具函式
            chat.tools = [
                tool for tool in chat.tools
                if tool['type'] != self.tool_name
            ]
        self.enabled = not self.enabled
        return True

    # 叫用單一函式並且將函式執行結果組成訊息後傳回
    def make_tool_msg(self, tool_call):
        tool_info = f'{tool_call.name}(**{tool_call.arguments})'
        if self.verbose: print(f'叫用：{tool_info}')
        func = eval(tool_call.name)
        args = json.loads(tool_call.arguments)
        result = func(**args)
        return {   # 建立可傳回函式執行結果的字典
            "type": "function_call_output", # 以工具角色送出回覆
            "call_id": tool_call.call_id, # 叫用函式的識別碼
            "output": result # 函式傳回值
        }

    def call_tools(self, tool_calls):
        msgs = []
        for tool_call in tool_calls:
            if tool_call.type == 'function_call':
                msgs.append(self.make_tool_msg(tool_call))
        return msgs if msgs else None

    def handle_event(self, chat, stream, event):
        if not self.enabled: return None
        if event.type != 'response.completed':
            return None
        # 呼叫函式
        tool_calls = event.response.output
        tool_results = self.call_tools(tool_calls)
        if tool_results: return tool_calls + tool_results
        return None

## 搭配指令處理器的 Chat 類別

In [ ]:
import pickle

class Chat:
    def __init__(self, client, **kwargs):
        self._client = client
        self._last_id = kwargs.pop('last_id', None)
        # 限制工具執行圈數，避免無窮盡叫用工具
        self._max_tools_rounds = kwargs.pop('max_tools_rounds', 4)
        self.tools = [] # 預設沒有使用工具
        self._commands = kwargs.pop('commands', [])

    def get_reply_text(self, msg, **kwargs) -> str:
        instructions = kwargs.pop('instructions', '使用繁體中文')
        model = kwargs.pop('model', 'gpt-4.1-nano')
        stream = kwargs.pop('stream', False)
        tool_results = [] # 函式叫用的相關資訊
        for command in self._commands:
            extra_args = {
                k: kwargs[k] + command.extra_args[k]
                if k in kwargs
                else command.extra_args[k]
                for k in command.extra_args
            }
            kwargs.update(extra_args)
        try:
            messages = [{'role': 'user', 'content': msg}]
            for _ in range(self._max_tools_rounds):
                # 方便稍後串接函式叫用資訊
                if tool_results: # 串接叫用函式的資訊
                    messages += tool_results
                response = self._client.responses.create(
                    instructions=instructions,
                    model=model,
                    input=messages,
                    stream=True, # 都以串流方式處理，簡化程式邏輯
                    previous_response_id=self._last_id, # 串接回應
                    **kwargs
                )
                for event in response:
                    for command in self._commands:
                        result = command.handle_event(
                            self, stream, event
                        )
                        if result: break
                    tool_results = []
                    if isinstance(result, list):
                        # 工具要送回給模型的訊息串列
                        tool_results = result
                        break
                    elif result: # 指令處理器產生的內容
                        yield result
                    if event.type == 'response.output_text.delta':
                        if stream: yield event.delta
                    elif event.type == (
                        'response.output_text.done'
                    ):
                        # 非串流模式要傳回完整內容
                        if not stream:
                            yield event.text
                    elif event.type == 'response.completed':
                        # 更新最後回應的識別碼
                        self._last_id = event.response.id
                if not tool_results:
                    # 沒有要送回給模型的訊息
                    # 表示已經成功生成內容
                    break
        except openai.APIError as err:
            print(f'Error:{err.body["message"]}', file=sys.stderr)
            return ''

    def find_tool_index(self, tool_type):
        for i, tool in enumerate(self.tools):
            if tool['type'] == tool_type: return i
        return -1

    def _get_prompt(self):
        prompt = ''
        for command in self._commands:
            idx = self.find_tool_index(command.tool_name)
            if idx != -1:
                prompt += f'{command.icon}'
        user_tools_count = len(self.tools) - len(prompt)
        prompt += f'(🛠️{user_tools_count})>>> '
        return prompt

    def _process_command(self, cmd):
        for command in self._commands:
            if command.handle_command(self, cmd):
                return True
        return False

    def loop(self, **kwargs) -> None:
        print("直接按 ↵ 可結束對話")
        while True:
            user_msg = input(self._get_prompt())
            if not user_msg.strip(): break # 直接 ↵ 就結束
            if self._process_command(user_msg):
                continue # 指令不需回覆，回頭讓使用者輸入
            text = ''
            display_handle = display(text, display_id=True)
            display_extra_handle = None
            for reply in self.get_reply_text(
                user_msg,
                tools=self.tools, # 傳入要使用的工具
                **kwargs
            ):
                if not isinstance(reply, str):
                    if not display_extra_handle:
                        display_extra_handle = display(
                            reply, display_id=True
                        )
                    else:
                        display_extra_handle.update(reply)
                    continue
                text += reply
                if os.path.exists(text):
                    display_handle.update(Markdown(f' text'))
                else:
                    display_handle.update(Markdown(text))

    def save(self, filename) -> None:
        with open(filename, 'wb') as f:
            pickle.dump(
                {
                    'last_id': self._last_id,
                    'tools': self.tools,
                    'commands': self._commands
                },
                f
            )

    def load(self, filename) -> None:
        with open(filename, 'rb') as f:
            data = pickle.load(f)
            self._last_id = data['last_id']
            self.tools = data['tools']
            self._commands = data['commands']

    def show_thread(self):
        if not self._last_id: return
        inputs = client.responses.input_items.list(self._last_id)
        response = client.responses.retrieve(self._last_id)
        for item in inputs.data[::-1]:
            # 略過函式叫用指示等非訊息內容
            if item.type != 'message': continue
            prompt = ">>> " if item.role == 'user' else ''
            for content in item.content:
                print(f'{prompt}{content.text}')
        print(response.output_text)

    def delete_thread(self):
        if not self._last_id: return
        last_id = self._last_id
        while last_id:
            response = client.responses.retrieve(last_id)
            last_id, curr_id = (
                response.previous_response_id,
                last_id
            )
            client.responses.delete(curr_id)
        self._last_id = None

### 測試工具與聊天

In [ ]:
commands = [
    WebSearchCommand(verbose=True),
    FileSearchCommand(),
    CodeInterpreterCommand(),
    ImageGenerationCommand(),
    FunctionCallingCommand(
        [shell_helper_tool],
        verbose=True
    )
]

In [ ]:
chat = Chat(
    client,
    commands=commands,
)

In [ ]:
chat.loop(
    model='gpt-4.1-mini',
    stream=True
)

### 測試儲存與還原

In [ ]:
chat.save('chat.pkl')

In [ ]:
chat1 = Chat(client)

In [ ]:
chat1.load('chat.pkl')

In [ ]:
chat1.loop(
    model='gpt-4.1-mini',
    stream=True
)

直接按 ↵ 可結束對話
🎨(🛠️0)>>> 


### 測試移除向量儲存區

In [ ]:
FileSearchCommand.remove_vector_store(chat)

### 測試顯示及移除討論串

In [ ]:
chat1.show_thread()

In [ ]:
chat1.delete_thread()